## 오토인코더

In [1]:
import torch
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.utils.data as data_utils
import torch.nn.functional as F
from tqdm import tqdm

from auto_encoder import AutoEncoder

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


### 매개변수 설정

In [3]:
num_epochs = 200
batch_size = 32
learning_rate = 3e-4
decay_factor = 0.99
model_save_path = "./auto_encoder_model.pth"

### 데이터 적재

In [4]:
train_data = datasets.MNIST(root='../data/',
                            train=True,
                            download=True,
                            transform=transforms.ToTensor())

indices = torch.arange(1000)
train_data_1k = data_utils.Subset(train_data, indices)
dataset_size = len(train_data_1k)

train_loader = DataLoader(dataset=train_data_1k,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=4,
                          pin_memory=True)

### 신경망 구조 정의

In [5]:
model = AutoEncoder().to(device)
model.train()

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.LambdaLR(optimizer,
                                        lr_lambda=(lambda epoch: decay_factor ** epoch))
print(model)

AutoEncoder(
  (encoder): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.01, inplace=True)
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (3): LeakyReLU(negative_slope=0.01, inplace=True)
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (5): LeakyReLU(negative_slope=0.01, inplace=True)
    (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (7): LeakyReLU(negative_slope=0.01, inplace=True)
    (8): Flatten(start_dim=1, end_dim=-1)
    (9): Linear(in_features=3136, out_features=2, bias=True)
  )
  (linear): Linear(in_features=2, out_features=3136, bias=True)
  (decoder): Sequential(
    (0): ConvTranspose2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.01, inplace=True)
    (2): ConvTranspose2d(64, 64, kernel_size=(4, 4), 

### 오토인코더 훈련

In [6]:
def r_loss(y_true, y_pred):
    return torch.mean(torch.mean(torch.square(y_true - y_pred), axis=[1, 2, 3]))

In [7]:
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            loss = F.mse_loss(outputs, inputs)
            loss.backward()
            optimizer.step()
            
        running_loss += loss.item() * inputs.size(0)
        
    scheduler.step()
    epoch_loss = running_loss / dataset_size
    print('Epoch {0:03d}\tLoss: {1:0.5f}'.format(epoch + 1, epoch_loss))

torch.save(model.state_dict(), model_save_path)

  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 001	Loss: 0.15063


  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 002	Loss: 0.07434


  0%|          | 0/32 [00:00<?, ?it/s]

...

Epoch 198	Loss: 0.03237


  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 199	Loss: 0.03233


100%|██████████| 32/32 [00:00<00:00, 103.58it/s]


Epoch 200	Loss: 0.03227
